In [1]:
# create env
# python3 -m venv prompt-gen-env
# source prompt-gen-env/bin/activate

!pip install langchain
!pip install langchain-community
!pip install langchain-huggingface
!pip install faiss-cpu  # or faiss-gpu if you have a compatible GPU
!pip install pandas
!pip install panel jupyter_bokeh

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
import faiss

In [4]:
import panel as pn
pn.extension()

# Set default styles for better appearance
pn.config.sizing_mode = "stretch_width"

# Dropdown for file selection - made wider
file_dropdown = pn.widgets.Select(
    name='Datei auswählen',
    options=[
        'elektromotoren-100.csv',
        'customers-100.csv'
    ],
    width=600  # Increased width
)

# Text display of selection
selected_file_display = pn.pane.Str(
    '', 
    styles={'font-size': '16px', 'color': 'green'},
    width=600  # Match width with dropdown
)

# Text input with conditional default - made wider
text_input = pn.widgets.TextInput(
    name='Suchanfrage',
    placeholder='Geben Sie Ihre Suchanfrage ein',
    width=600  # Same width as other components
)

# Callback function for selection changes
def update_display(event):
    selected_file_display.object = f"Ausgewählte Datei: {event.new}"
    
    # Update text input based on selection
    if event.new == 'elektromotoren-100.csv':
        text_input.value = 'Wann wurde der Motor mit Config ID CFG345 eingebaut?'
    else:
        text_input.value = 'Which company does Sheryl Baxter work for?'

file_dropdown.param.watch(update_display, 'value')

# Initialize with default value
if file_dropdown.value == 'elektromotoren-100.csv':
    text_input.value = 'Wann wurde der Motor mit Config ID CFG345 eingebaut?'
else:
    text_input.value = 'Which company does Sheryl Baxter work for?'

# Create layout with some additional styling
layout = pn.Column(
    pn.Row(file_dropdown, margin=(0, 0, 20, 0)),
    pn.Row(selected_file_display, margin=(0, 0, 20, 0)),
    pn.Row(text_input),
    width=650,  # Container width
    margin=(20, 20, 20, 20)  # Add some margin around the container
)

# Display
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'bab38baf-5771-4396-b10d-780785327128': {'version…

In [14]:
file_path = file_dropdown.value
question = text_input.value
data = pd.read_csv(file_path)
data.head()

# Load documents
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

# Use BGE Embeddings
if file_path == "elektromotoren-100.csv":
    embedding_model_name = "BAAI/bge-m3" # mulitlingual
else:
    embedding_model_name = "BAAI/bge-base-en"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Create FAISS index
dim = len(embeddings.embed_query("Hello world"))
index = faiss.IndexFlatL2(dim)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)
vector_store.add_documents(documents=docs)
doc_id = list(vector_store.docstore._dict.keys())[0]
print(doc_id)
print(vector_store.docstore._dict[doc_id].page_content)

# ---- STEP 1: Set up the prompt template ----
if file_path == "elektromotoren-100.csv":
    system_prompt = (
        "Du bist ein Assistent zur Analyse und Auswertung von Daten zu Elektromotoren. "
        "Nutze die folgenden Teile des abgerufenen Tracing-Datensatzes, um die gestellte "
        "Frage zu beantworten. Wenn du die Antwort nicht sicher weißt, gib an, dass du es "
        "nicht weißt. Antworte in maximal drei Sätzen und halte die Antwort präzise und sachlich."
        "\n\n"
        "{context}"
    )


    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])
else:
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])

# ---- STEP 2: Retrieve documents according to question ----
retriever = vector_store.as_retriever()
docs = retriever.invoke(question)

# ---- STEP 3: Fill the prompt ----
filled_prompt = prompt.format(
    context="\n\n".join([doc.page_content for doc in docs]),
    input=question
)

print(filled_prompt)

2f2ab5f7-d7ff-4bbd-a8dd-d5d9bbcb4cd2
Index: 1
Motor_ID: EM2025000
Config_ID: CFG345
Spannung_V: 400
Leistung_kW: 44.6
Drehmoment_Nm: 39.2
Temp_Bereich_C: -25 – 118
Einbaudatum: 2024-07-06
Qualitaetsprüfung: nein
Fertigungsstraße: Linie C
System: Du bist ein Assistent zur Analyse und Auswertung von Daten zu Elektromotoren. Nutze die folgenden Teile des abgerufenen Tracing-Datensatzes, um die gestellte Frage zu beantworten. Wenn du die Antwort nicht sicher weißt, gib an, dass du es nicht weißt. Antworte in maximal drei Sätzen und halte die Antwort präzise und sachlich.

Index: 20
Motor_ID: EM2025019
Config_ID: CFG825
Spannung_V: 480
Leistung_kW: 1.8
Drehmoment_Nm: 37.4
Temp_Bereich_C: -3 – 91
Einbaudatum: 2024-07-19
Qualitaetsprüfung: nein
Fertigungsstraße: Linie B

Index: 25
Motor_ID: EM2025024
Config_ID: CFG718
Spannung_V: 480
Leistung_kW: 30.7
Drehmoment_Nm: 376.9
Temp_Bereich_C: -30 – 95
Einbaudatum: 2025-03-30
Qualitaetsprüfung: nein
Fertigungsstraße: Linie B

Index: 58
Motor_ID: EM

# Same Code as in cell above but step by step

In [15]:
file_path = file_dropdown.value
question = text_input.value
print(file_path)
print(question)
data = pd.read_csv(file_path)
data.head()

customers-100.csv
Which company does Sheryl Baxter work for?


,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


In [16]:
# Load documents
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()
docs[0:3]

[Document(metadata={'source': 'customers-100.csv', 'row': 0}, page_content='Index: 1\nCustomer Id: DD37Cf93aecA6Dc\nFirst Name: Sheryl\nLast Name: Baxter\nCompany: Rasmussen Group\nCity: East Leonard\nCountry: Chile\nPhone 1: 229.077.5154\nPhone 2: 397.884.0519x718\nEmail: zunigavanessa@smith.info\nSubscription Date: 2020-08-24\nWebsite: http://www.stephenson.com/'),
 Document(metadata={'source': 'customers-100.csv', 'row': 1}, page_content='Index: 2\nCustomer Id: 1Ef7b82A4CAAD10\nFirst Name: Preston\nLast Name: Lozano\nCompany: Vega-Gentry\nCity: East Jimmychester\nCountry: Djibouti\nPhone 1: 5153435776\nPhone 2: 686-620-1820x944\nEmail: vmata@colon.com\nSubscription Date: 2021-04-23\nWebsite: http://www.hobbs.com/'),
 Document(metadata={'source': 'customers-100.csv', 'row': 2}, page_content='Index: 3\nCustomer Id: 6F94879bDAfE5a6\nFirst Name: Roy\nLast Name: Berry\nCompany: Murillo-Perry\nCity: Isabelborough\nCountry: Antigua and Barbuda\nPhone 1: +1-539-402-0259\nPhone 2: (496)978-3

In [17]:
# Use BGE Embeddings
if file_path == "elektromotoren-100.csv":
    embedding_model_name = "BAAI/bge-m3" # mulitlingual
else:
    embedding_model_name = "BAAI/bge-base-en"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

In [18]:
# Create FAISS index
dim = len(embeddings.embed_query("Hello world"))
print("embeddings dimension:", dim)
index = faiss.IndexFlatL2(dim)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

embeddings dimension: 768


In [19]:
vector_store.add_documents(documents=docs)
doc_id = list(vector_store.docstore._dict.keys())[0]
print(doc_id)
print(vector_store.docstore._dict[doc_id].page_content)

b653e141-07d4-4cea-b165-4348c1008b0f
Index: 1
Customer Id: DD37Cf93aecA6Dc
First Name: Sheryl
Last Name: Baxter
Company: Rasmussen Group
City: East Leonard
Country: Chile
Phone 1: 229.077.5154
Phone 2: 397.884.0519x718
Email: zunigavanessa@smith.info
Subscription Date: 2020-08-24
Website: http://www.stephenson.com/


In [20]:
# # Use Mistral 7B via ctransformers
# llm = CTransformers(
#     model='TheBloke/Mistral-7B-Instruct-v0.1-GGUF',  # GGUF quantized
#     # model_file='mistral-7b-instruct-v0.1.Q4_K_M.gguf',
#     config={'max_new_tokens': 10000, 'temperature': 0.7}
# )

In [21]:
# ---- STEP 1: Set up the prompt template ----
if file_path == "elektromotoren-100.csv":
    system_prompt = (
        "Du bist ein Assistent zur Analyse und Auswertung von Daten zu Elektromotoren. "
        "Nutze die folgenden Teile des abgerufenen Tracing-Datensatzes, um die gestellte "
        "Frage zu beantworten. Wenn du die Antwort nicht sicher weißt, gib an, dass du es "
        "nicht weißt. Antworte in maximal drei Sätzen und halte die Antwort präzise und sachlich."
        "\n\n"
        "{context}"
    )


    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])
    
    # question = "Wann wurde der Motor mit Config ID CFG345 eingebaut?"
else:
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])


    # question = "which company does Sheryl Baxter work for?"

# ---- STEP 2: Retrieve documents according to question ----
retriever = vector_store.as_retriever()
docs = retriever.invoke(question)

# # ---- STEP 3: Preview the retrieved context ----
# print("Retrieved documents:\n")
# for i, doc in enumerate(docs):
#     print(f"[{i+1}] {doc.page_content}\n")

# ---- STEP 4: Fill the prompt manually ----
filled_prompt = prompt.format(
    context="\n\n".join([doc.page_content for doc in docs]),
    input=question
)

print("\n\n==== Final Prompt Sent to the LLM ====\n")
print(filled_prompt)

# # ---- STEP 5: Run the LLM with the filled prompt ----

# response = llm.invoke(filled_prompt)
# print("\n\n==== LLM Answer ====\n")
# print(response)



==== Final Prompt Sent to the LLM ====

System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

Index: 1
Customer Id: DD37Cf93aecA6Dc
First Name: Sheryl
Last Name: Baxter
Company: Rasmussen Group
City: East Leonard
Country: Chile
Phone 1: 229.077.5154
Phone 2: 397.884.0519x718
Email: zunigavanessa@smith.info
Subscription Date: 2020-08-24
Website: http://www.stephenson.com/

Index: 9
Customer Id: C2dE4dEEc489ae0
First Name: Sheryl
Last Name: Meyers
Company: Browning-Simon
City: Robersonstad
Country: Cyprus
Phone 1: 854-138-4911x5772
Phone 2: +1-448-910-2276x729
Email: mariokhan@ryan-pope.org
Subscription Date: 2020-01-13
Website: https://www.bullock.net/

Index: 70
Customer Id: CC68FD1D3Bbbf22
First Name: Riley
Last Name: Good
Company: Wade PLC
City: Erikaville
Country: Canada
Phone 1: 6977745822
Phone